In [ ]:
#!pip install -U lime

import pandas as pd
import numpy  as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from tabulate import tabulate
from texttable import Texttable

from plotly.subplots import make_subplots

In [ ]:
base1 = pd.read_csv("C:/Users/jaque/Desktop/Behavior Score/Bases/AmostraBehavior.csv", sep = ";")
base1["Intercept"] = 1
base1.head()

---

<center><strong><font color = "darkblue" size=4> Preparação ABT modelagem </font></strong></center>

---

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
labels = base1.columns[2:len(base1.columns)]

X = base1[labels] # Covariáveis
y = base1.Target # Target

# Base dados balanceados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 999)
X_val, X_test, y_val, y_test     = train_test_split(X_test, y_test, test_size = 0.5, random_state = 999)

print(y_train.count(), y_test.count(),y_val.count())

---

<center><strong><font color = "darkblue" size=4> Balanceamento da informação</font></strong></center>

---

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

under = RandomUnderSampler(sampling_strategy=0.5)
X_under_train, y_under_train = under.fit_resample(X_train, y_train)
tab2  = pd.crosstab(index = y_under_train, columns = ["count"]) 
tab2["percent"] = (tab2/tab2.sum())*100
tab2

In [ ]:
# Comparando as respostas

tab_under  = pd.crosstab(index = y_under_train, columns = ["count"]) 
tab_under["percent_under"] = (tab_under/tab_under.sum())*100
tab_under = tab_under.reset_index()

tab_orig  = pd.crosstab(index = y_train, columns = ["count"])
tab_orig["percent_orig"] = (tab_orig/tab_orig.sum())*100
tab_orig = tab_orig.reset_index()


fig_comp = make_subplots(rows=1, cols=2)

trace0 = go.Bar(x = ["Bom","Mau"], y = tab_under["percent_under"], name = "Balanceado", marker_color = "#0066cc")
trace1 = go.Bar(x = ["Bom","Mau"], y = tab_orig["percent_orig"], name = "DesBalanceado", marker_color = "#0099cc")

fig_comp.append_trace(trace0,  1, 1)
fig_comp.append_trace(trace1,  1, 2)

fig_comp.update_layout(height = 400, width = 900, title = "Comparação entre target balanceado e desbalanceado")

fig_comp.show()


In [ ]:
X_under_train.describe()

In [ ]:
X_train.describe()

---

<center><strong><font color = "darkblue" size=4> Funções de Avaliação dos modelos </font></strong></center>

---

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn import metrics
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt

### Descrição das medidas de avaliação

TP: Classificação correta da classe Positiva;<br>

FN (Erro Tipo II): erro em que o modelo previu a classe Negativo quando o real era Positiva;<br>

FP (Erro Tipo I): erro em que o modelo previu a classe Positiva quando o real era Negativa;<br>

TN: Classificação correta da classe Negativa.<br>
<br>

- Acurácia [0,1]: É a proporção de predições corretas, sem considerar o que é positivo e o que negativo e sim o acerto total. É dada por: TP (Verdadeiro positivo) + TN (Verdadeiro negativo). 
<br>

- Precisão [0,1]: Do total de classificados positivos, quantos efetivamente eram: TP/(FP+TP), ou seja, quanto do evento de interesse estou acertando.
<br>

- Recall [0,1]: Do total de observados que são meu evento de interessem quantos efetivamente são: TP/(FN+TP), isto é, quando o modelo deve de qualquer maneira encontrar o evento de interesse, mesmo que classifique alguns não eventos como positivos (situação de Falso Positivo) no processo, deve-se considerar tal medida. 
<br>

- AUC [0,1]: O AUC representa a área da crva ROC e mede a capacidade de separabilidade do modelo. Quanto maior o AUC, melhor o modelo está em prever 0 como 0 e 1 como 1. 
<br>

- Gini [0,1]: O Coeficiente de Gini consiste em um número entre 0 e 1, e que 0 corresponde à completa igualdade e 1 corresponde à completa desigualdade entre os eventos comparados, sendo obtido por Gini = 2*AUC-1; 
<br>

- LogLoss [-inf,1]: A perda logarítmica (relacionada à entropia cruzada) mede o desempenho de um modelo de classificação em que a entrada de previsão é um valor de probabilidade entre 0 e 1. Verificando a vantagem do classificador sobre uma previsão aleatória. Logloss = [-1/n] sum(n)sum(m)[y*log(p)]
<br>

- KS [0,1]: Técnica não paramétrica para determinar se duas amostras foram extraídas de populações com distribuições similares. Dada por KS = max|F0(y) - F1(y)|.
<br>

- ASE: Média dos erros de predição ao quadrado, a medida não tem interpretação direta sozinha, mas pode ser utilizada para comparação de modelos, sendo o modelos com menor ASE, o que apresenta o menor erro médio de classificação. 
<br>


In [ ]:
# Função para diagnóstico dos modelos
#https://stackoverflow.com/questions/53846943/sklearn-logistic-regression-adjust-cutoff-point

def AVALIA_MOD(modelo, y_prob, y_obs, titulo, base, corte):
    
    pred_Y_test = {}
    pred_Y_data = pd.DataFrame(modelo.predict_proba(y_prob)[:,1], columns=["y_prob"])
    pred_Y_data["y_hat"] = modelo.predict(y_prob)
    obs_Y_data = y_obs.reset_index()
    pred_data = pd.concat([obs_Y_data,
                         pred_Y_data], axis = 1)
    
    pred_data["erro2"]     = (pred_data.Target - pred_data.y_prob)**2
    pred_data["y_hat_new"] = np.where(pred_data.y_prob > corte, 1, 0)
    
    print("--------------------------------------------")
    print("----------------",base,"--------------------")
    print("\n")
    KS = ks_2samp(pred_data.loc[pred_data.Target == 0, "y_prob"], pred_data.loc[pred_data.Target == 1, "y_prob"])
    KS = round(KS[0],2)*100
    
    ASE = round(np.mean(pred_data.erro2),2)
    
    AUC = roc_auc_score(pred_data.Target,pred_data.y_prob)
    GINI = 2*AUC - 1
    AUC = round(AUC, 2)*100

    # Curva Roc
    fpr, tpr, thresholds  = roc_curve(pred_data.Target,pred_data.y_prob)
    
    trace1 = go.Scatter(x = fpr, y = tpr, 
                        mode = "lines", 
                        line = dict(color = "#b30000", width = 2),
                        name = 'AUC: %0.2f' % AUC)
    trace2 = go.Scatter(x=[0, 1], y=[0, 1], 
                        mode='lines', 
                        line=dict(color = "navy", width=2, dash = "dash"),
                        showlegend=False)
    layout = go.Layout(xaxis=dict(title = "FP"),
                       yaxis=dict(title = "TP"))
    plot_roc = go.Figure(data=[trace1, trace2], layout=layout)
    plot_roc.update_layout(title_text = "Curva ROC "+titulo)
    
    # Matriz de confusão
    cm = confusion_matrix(pred_data.Target, pred_data.y_hat)
    conf_m = cm/np.sum(cm)*100
    TP = round(conf_m[0,0],2)
    FP = round(conf_m[0,1],2)
    FN = round(conf_m[1,0],2)
    TN = round(conf_m[1,1],2)
    
    cm2 = confusion_matrix(pred_data.Target, pred_data.y_hat_new)
    conf_m2 = cm2/np.sum(cm2)*100
    acur_cut = conf_m2[0,0] +conf_m2[1,1]
    
    print("Matriz de confusão default",
         "\n","|","TP", TP,"|","FP",FP,"|","\n","|","FN", FN,"|","TN", TN,"|")
    print("\n")
    
    print("Matriz de confusão novo cutoff",
         "\n", conf_m2)
    print("\n")
    
    CONCORD = round(TP + TN ,2)
    PRECS   = TP/(FP+TP)
    RECALL  = TP/(FN+TP)
                           
    LogLoss = log_loss(pred_data.Target, pred_data.y_hat)
    
    d = {"Metricas": ["Acurácia","Precisão","Recall","LogLoss","AUC","KS","Gini","ASE"],
         "Valores": [CONCORD,PRECS,RECALL,LogLoss,AUC, KS,GINI,ASE]}
    
    # Base de medidas de qualidade dos ajuste
    METRICAS = pd.DataFrame(data=d)
    
    # Base para gráfico de ordenação
    pred_data = pred_data.sort_values(by=["y_prob"])

    pred_data["FX_yprob"] = pd.qcut(pred_data.y_prob, 10)
    pred_data["Default"]  = np.mean(pred_data.Target)

    ordena = pred_data.groupby("FX_yprob")[["Target","Default"]].mean().reset_index()   
    ordena["FX_yprob2"] = lab = np.arange(10)
    ordena["LIFT"] = ordena.Target/ordena.Default

#--------------------------------------------------------   
    lab1 = titulo[7:]

    if lab1 == "WOE Balanc":
        col = "#5c5c8a"
    elif lab1 == "WOE Desbalanc":
        col = "#cc0000"
    elif lab1 == "Desbalanc":
        col = "#008ae6"     
    else:
        col = "#8a8a5c" 
#----------------------------------------------------------    
    # Gráfico de ordenação
    fig_ord = data=go.Scatter(x = ordena.FX_yprob2, y = ordena.LIFT, name = "Lift "+titulo, marker=dict(color = col))
#    fig_ord.update_layout(title_text = "Ordenação: Lift "+titulo)
    
    print("Acurácia:", CONCORD,"Acurácia cutoff:",round(acur_cut,2),"|","Precisão:", round(PRECS,2),"|","Recall:", round(RECALL,2))
    print("\n")
    print("KS:", KS,"|", "AUC:",AUC,"|","GINI:",round(GINI,2), "|","LogLoss:",round(LogLoss,2), "|","ASE:", ASE)
    print("\n")
    return(pred_data,ordena,METRICAS,fig_ord,plot_roc,titulo,lab1,AUC)

In [ ]:
# Função para avaliar importância das variáveis/features
def ML_IMPORTANCIA(modelo, feat):
    importances = modelo.feature_importances_
    indices = np.argsort(importances)
    indices = pd.Series(indices)
    features = pd.Series(feat)
    
    print("    Results: Importância das Variáveis")
    print("============================================")
    tt = pd.Series(importances[indices],
                 index=features[indices]).sort_values(ascending=False)
    print(tt)
    print("============================================")   

In [ ]:
# Função para calcular o shift da AUC, ou seja, avaliar se ocorreram quedas na medida em amostras distintas
def SHIFT_AUC(x,y):
    shift = abs(round((x[7] - y[7])/y[7],2)*100)
    z = list(x)
    z[2] = x[2].append({"Metricas" : "Shift AUC" , "Valores" : shift} , ignore_index=True)
    x = tuple(z)
    return(x)
    print("Shift AUC:",shift)


---

<center><strong><font color = "darkblue" size=4> Regressão Logística </font></strong></center>

---

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm


In [ ]:
# http://prorum.com/?qa=3254/classificacao-usando-stepwise-logistic-regression-usando
# https://www.kdnuggets.com/2018/06/step-forward-feature-selection-python.html
base1.columns

<center><strong><font color = "darkblue" size=4>------------------------------------------ Logit: Balanceados por WOE ------------------------------------------</font></strong></center>

In [ ]:
# Balanceados por WOE
X_WOE2 = ["Intercept","WOE_CAT_Idade","WOE_N_Atraso60_89Dias","WOE_N_atrasos_Ult90Dias",
          "WOE_CAT_UltPercLimit","WOE_N_emprestimos"]

# https://www.statsmodels.org/stable/glm.html
M_logit_under_WOE  = sm.GLM(y_under_train,  X_under_train[X_WOE2], family = sm.families.Binomial(), ).fit()
M_logit_under_WOE2 = LogisticRegression().fit(X_under_train[X_WOE2[1:6]], y_under_train)

print(M_logit_under_WOE.summary2())

# Modelo balanceado com WOE
logit_under_WOE_test = AVALIA_MOD(modelo = M_logit_under_WOE2, y_prob = X_test[X_WOE2[1:6]], y_obs = y_test, base = "Teste", titulo = "Logit: WOE Balanc", corte = 0.3)
logit_under_WOE_val  = AVALIA_MOD(modelo = M_logit_under_WOE2, y_prob = X_val[X_WOE2[1:6]], y_obs = y_val,  base = "Valida" , titulo = "Logit: WOE Balanc", corte = 0.3)
    
logit_under_WOE_test = SHIFT_AUC(x = logit_under_WOE_test, y = logit_under_WOE_val)

<center><strong><font color = "darkblue" size=4> ------------------------------------------ Logit: Não Balanceados por WOE --------------------------------------</font></strong></center>

In [ ]:
#Não Balanceados por WOE

M_logit_WOE  = sm.GLM(y_train,  X_train[X_WOE2], family=sm.families.Binomial()).fit()
M_logit_WOE2 = LogisticRegression().fit(X_train[X_WOE2[1:6]], y_train)

print(M_logit_WOE.summary2())

# Modelo Não balanceado com WOE

logit_WOE_test = AVALIA_MOD(modelo = M_logit_WOE2, y_prob = X_test[X_WOE2[1:6]],  y_obs = y_test, base="Teste", titulo = "Logit: WOE Desbalanc",  corte = np.mean(y_test))
logit_WOE_val  = AVALIA_MOD(modelo = M_logit_WOE2, y_prob = X_val[X_WOE2[1:6]], y_obs = y_val, base="Valida", titulo = "Logit: WOE Desbalanc",corte = np.mean(y_val))

logit_WOE_test = SHIFT_AUC(x = logit_WOE_test, y = logit_WOE_val)


<center><strong><font color = "darkblue" size=4> ------------------------------------------ Logit: Balanceados por Contínuas -------------------------------------- </font></strong></center>

In [ ]:
# Modelo Balanceado covariáveis continuas
# https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

XX = ["Intercept",'UltPercLimit', 'Idade', 'N_Atraso30_59Dias', 'N_EmeprestimosAbertos',
                'N_atrasos_Ult90Dias', 'N_Atraso60_89Dias']

M_logit_under = sm.GLM(y_under_train,  X_under_train[XX], family=sm.families.Binomial(),max_iter = 5000).fit()
M_logit_under2 = LogisticRegression().fit(X_under_train[XX[1:6]], y_under_train)

print(M_logit_under.summary2())

# Modelo  balanceado com Contínuas
logit_under_test = AVALIA_MOD(modelo = M_logit_under2, y_prob = X_test[XX[1:6]], y_obs = y_test, base="Teste", titulo = "Logit: Balanc", corte = np.mean(y_test))
logit_under_val   = AVALIA_MOD(modelo = M_logit_under2, y_prob = X_val[XX[1:6]], y_obs = y_val, base="Valida", titulo = "Logit: Balanc", corte = np.mean(y_val))

logit_under_test = SHIFT_AUC(x = logit_under_test, y = logit_under_val)


<center><strong><font color = "darkblue" size=4> ------------------------------------------ Logit: Não Balanceados por Contínuas -------------------------------------- </font></strong></center>

In [ ]:
# Modelo Não Balanceado covariáveis continuas

M_logit = sm.GLM(y_train,  X_train[XX], family=sm.families.Binomial(),max_iter = 5000).fit()
M_logit2 = LogisticRegression().fit(X_train[XX[1:6]], y_train)

print(M_logit.summary2())

# Modelo balanceado com Contínuas
pred_Y_test = {}
logit_test = AVALIA_MOD(modelo = M_logit2, y_prob = X_test[XX[1:6]], y_obs = y_test, titulo = "Logit: Desbalanc", base="Teste",corte = np.mean(y_test))
logit_val  = AVALIA_MOD(modelo = M_logit2, y_prob = X_val[XX[1:6]], y_obs = y_val, titulo = "Logit: Desbalanc", base="Treino",corte = np.mean(y_val))

logit_test = SHIFT_AUC(x = logit_test, y = logit_val)


---

<center><strong><font color = "darkblue" size=4> Random Forest </font></strong></center>

---

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#https://towardsdatascience.com/random-forest-in-python-24d0893d51c0
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

features = ['UltPercLimit', 'Idade', 'N_Atraso30_59Dias',
       'RendaMensal', 'N_EmeprestimosAbertos',
       'N_atrasos_Ult90Dias', 'N_emprestimos', 'N_Atraso60_89Dias',
       'N_dependentes', 'lnRazaoGastos']


<center><strong><font color = "darkblue" size=4> -----------------  RF: Seleção de Hiperparâmetros -------------------</font></strong></center>

### Principais Parâmetros do Radom Forest

- n_estimators: O número de árvores empregadas no algoritmo
<br>

- max_features: O número de variáveis amostradas em cada árvore, normalmente usamos sqrt(p), log2(p)
<br>

- max_depth: É a profundidade da árvore, quanto maior seu valor mais complexa é a árvore, podendo decorar o conjunto de treino (overfitting), o que pode degradar seu poder preditivo quando aplicado a novos dados. Isso pode ser mitigado "podando" a árvore de decisão ao atribuir uma profundidade máxima ou uma quantidade máxima de folhas.
<br>

- min_samples_split: O volume mínimo considerado antes de dividir um nó interno
<br>

- min_samples_leaf: O volume mínimo de dados permitidos em uma folha
<br>

- bootstrap: Método de amostragem das linhas(população). Se False, o conjunto de dados inteiro será usado para criar cada árvore, o que pode gerar um problema de árvores altamento correlacionadas.
<br>

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 900, num = 6)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 100, num = 5)]
max_depth.append(None)

min_samples_split = [50, 200, 500]

min_samples_leaf = [10, 20, 50]

bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_grid

In [ ]:
# Criando o modelo base
rf = RandomForestClassifier()
# Utilizando 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=999, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_under_train[features], y_under_train)
rf_random.best_params_

<center><strong><font color = "darkblue" size=4> -----------------  RF: Dados contínuos Não balanceados -------------------</font></strong></center>

In [ ]:
rf = RandomForestClassifier(n_estimators = 480, random_state = 999, min_samples_split = 10, 
                            min_samples_leaf = 4, max_features = "auto", max_depth = 10, 
                                bootstrap = True).fit(X_train[features], y_train)

ML_IMPORTANCIA(rf, features)

# Modelo Não balanceado com Contínuas
rf_test = AVALIA_MOD(modelo = rf, y_prob = X_test[features], y_obs = y_test, titulo = "Randf: Desbalanc", base="Teste", corte = np.mean(y_test))
rf_val  = AVALIA_MOD(modelo = rf, y_prob = X_val[features],  y_obs = y_val,  titulo = "Randf: Desbalanc",  base="Valida", corte = np.mean(y_val))

rf_test = SHIFT_AUC(x = rf_test, y = rf_val)

<center><strong><font color = "darkblue" size=4> -----------------  RF: Dados contínuos balanceados -------------------</font></strong></center>

In [ ]:
rf_under = RandomForestClassifier(n_estimators = 480, random_state = 999, min_samples_split = 10, 
                            min_samples_leaf = 4, max_features = "auto", max_depth = 10, 
                                bootstrap = True).fit(X_under_train[features], y_under_train)

ML_IMPORTANCIA(rf_under, features)

# Modelo Não balanceado com Contínuas
rf_test_under = AVALIA_MOD(modelo = rf_under, y_prob = X_test[features], y_obs = y_test, titulo = "Randf: Balanc", base="Teste", corte = 0.3)
rf_val_under  = AVALIA_MOD(modelo = rf_under, y_prob = X_val[features],  y_obs = y_val,  titulo = "Randf: Balanc", base="Valida", corte = 0.3)

rf_test_under = SHIFT_AUC(x = rf_test_under, y = rf_val_under)

<center><strong><font color = "darkblue" size=4> -----------------  RF: Dados WOE Não balanceados -------------------</font></strong></center>

In [ ]:
X_WOE3 = ["WOE_CAT_Idade","WOE_N_Atraso60_89Dias","WOE_N_atrasos_Ult90Dias",
          "WOE_CAT_UltPercLimit","WOE_N_emprestimos"]
    
rf_WOE = RandomForestClassifier(n_estimators = 480, random_state = 999, min_samples_split = 10, 
                            min_samples_leaf = 4, max_features = "auto", max_depth = 10, 
                                bootstrap = True).fit(X_train[X_WOE3], y_train)

ML_IMPORTANCIA(rf_WOE, X_WOE3)

# Modelo Não balanceado com Contínuas
rf_test_WOE = AVALIA_MOD(modelo = rf_WOE, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "Randf: WOE Desbalanc", base="Teste",corte = np.mean(y_test))
rf_val_WOE  = AVALIA_MOD(modelo = rf_WOE, y_prob = X_val[X_WOE3],  y_obs = y_val, titulo = "Randf: WOE Desbalanc",  base="Valida",corte = np.mean(y_val))

rf_test_WOE = SHIFT_AUC(x = rf_test_WOE, y = rf_val_WOE)

<center><strong><font color = "darkblue" size=4> -----------------  RF: Dados WOE balanceados -------------------</font></strong></center>

In [ ]:
rf_WOE_under = RandomForestClassifier(n_estimators = 480, random_state = 999, min_samples_split = 10, 
                            min_samples_leaf = 4, max_features = "auto", max_depth = 10, 
                                bootstrap = True).fit(X_under_train[X_WOE3], y_under_train)

ML_IMPORTANCIA(rf_WOE_under, X_WOE3)

# Modelo Não balanceado com Contínuas
rf_test_under_WOE = AVALIA_MOD(modelo = rf_WOE, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "Randf: WOE Balanc", base="Teste",corte = 0.3)
rf_val_under_WOE  = AVALIA_MOD(modelo = rf_WOE, y_prob = X_val[X_WOE3],  y_obs = y_val, titulo = "Randf: WOE Balanc",  base="Valida",corte = 0.3)

rf_test_under_WOE = SHIFT_AUC(x = rf_test_under_WOE, y = rf_val_under_WOE)

---

<center><strong><font color = "darkblue" size=4> Gradiet boosting </font></strong></center>

---

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

### Principais Parâmetros do Gradient Boosting

- learning_rate: Determina o impacto de cada árvore no resultado final. O GBM funciona com uma estimativa inicial que é atualizada usando a saída de cada árvore. O parâmetro de learning controla a magnitude dessa mudança nas estimativas. Valores mais baixos são geralmente preferidos, pois tornam o modelo robusto às características específicas da árvore e, assim, permite que ele se generalize bem. Valores mais baixos exigem um número maior de árvores para modelar todas as relações sendo computacionalmente exaustivos.
<br>

- n_estimators: O número de árvores sequenciais a serem modeladas. O GB é bastante robusto contra o overfitting, portanto um grande número geralmente resulta em melhor desempenho.
<br>

- max_depth: LTime_serieimita o número de nós na árvore. Ajuste esse parâmetro para obter o melhor desempenho; o melhor valor depende da interação das variáveis de entrada
<br>

- subsample: A parcela de observações selecionadas para cada árvore. A seleção é feita por amostragem aleatória. Valores ligeiramente inferiores a 1 tornam o modelo robusto, reduzindo a variação. Valores próximos a 0.8 geralmente funcionam bem.
<br>

- loss:  Refere-se à função de perda a ser minimizada em cada divisão. Geralmente os valores default funcionam bem. 


<center><strong><font color = "darkblue" size=4> -----------------  GB: Seleção de Hiperparâmetros -------------------</font></strong></center>

In [ ]:
# https://www.datacareer.de/blog/parameter-tuning-in-gradient-boosting-gbm/

test3 = { 'learning_rate':[0.1, 0.05, 0.01, 0.005], 
            'n_estimators':[100, 250, 500, 1000, 1500],
            'max_depth':[2, 3, 5, 6] }

tuning = GridSearchCV(estimator = GradientBoostingClassifier(min_samples_split = 2, min_samples_leaf = 1, 
                                                             subsample = 1, max_features = "sqrt", random_state = 999), 
            param_grid = test3, scoring = "accuracy", n_jobs = 4, iid = False, cv = 5)

tuning.fit(X_under_train[features], y_under_train)
tuning.best_params_

<center><strong><font color = "darkblue" size=4> -----------------  GB: Dados contínuos Balanceados -------------------</font></strong></center>

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

clf_under = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 1000,
                                       random_state = 999).fit(X_under_train[features], y_under_train)

ML_IMPORTANCIA(clf_under, features)

# Balanceados contínuos
gb_test_under = AVALIA_MOD(modelo = clf_under, y_prob = X_test[features], y_obs = y_test, titulo = "GBoos: Balanc", base = "Teste", corte = 0.3)
gb_val_under  = AVALIA_MOD(modelo = clf_under, y_prob = X_val[features],  y_obs = y_val,  titulo = "GBoos: Balanc", base = "Valida", corte = 0.3)

gb_test_under = SHIFT_AUC(x = gb_test_under, y = gb_val_under)

<center><strong><font color = "darkblue" size=4> -----------------  GB: Dados contínuos Não Balanceados -------------------</font></strong></center>

In [ ]:
clf = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 1000,
                                       random_state = 999).fit(X_train[features], y_train)

ML_IMPORTANCIA(clf, features)

# Não balanceados contínuos
gb_test = AVALIA_MOD(modelo = clf, y_prob = X_test[features], y_obs = y_test, titulo = "GBoos: Desbalanc", base = "Teste", corte = 0.3)
gb_val  = AVALIA_MOD(modelo = clf, y_prob = X_val[features],  y_obs = y_val,  titulo = "GBoos: Desbalanc", base = "Valida", corte = 0.3)

gb_test = SHIFT_AUC(x = gb_test, y = gb_val)

<center><strong><font color = "darkblue" size=4> -----------------  GB: Dados WOE Não Balanceados -------------------</font></strong></center>

In [ ]:
clf_woe = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 1000,
                                       random_state = 999).fit(X_train[X_WOE3], y_train)

ML_IMPORTANCIA(clf_woe, X_WOE3)

# Não balanceados WOE
gb_WOE_test = AVALIA_MOD(modelo = clf_woe, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "GBoos: WOE Desbalanc", base = "Teste", corte = 0.3)
gb_WOE_val  = AVALIA_MOD(modelo = clf_woe, y_prob = X_val[X_WOE3],  y_obs = y_val,  titulo = "GBoos: WOE Desbalanc", base = "Valida", corte = 0.3)

gb_WOE_test = SHIFT_AUC(x = gb_WOE_test, y = gb_WOE_val)

<center><strong><font color = "darkblue" size=4> -----------------  GB: Dados WOE Balanceados -------------------</font></strong></center>

In [ ]:
clf_woe_under = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 1000,
                                       random_state = 999).fit(X_under_train[X_WOE3], y_under_train)

ML_IMPORTANCIA(clf_woe_under, X_WOE3)

# Não balanceados WOE
gb_under_WOE_test = AVALIA_MOD(modelo = clf_woe_under, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "GBoos: WOE Balanc", base = "Teste", corte = 0.3)
gb_under_WOE_val  = AVALIA_MOD(modelo = clf_woe_under, y_prob = X_val[X_WOE3],  y_obs = y_val,  titulo = "GBoos: WOE Balanc", base = "Valida", corte = 0.3)

gb_under_WOE_test = SHIFT_AUC(x = gb_under_WOE_test, y = gb_under_WOE_val)

---

<center><strong><font color = "darkblue" size=4> Adaboost </font></strong></center>

---

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets

### Principais Parâmetros do AdaBoost

- learning_rate: Taxa de aprendizado que controla a contribuição de cada modelo para a previsão do conjunto. Por padrão, é definido como 1 ou contribuição total. Valores menores ou maiores podem ser adequados, dependendo do número de modelos usados no conjunto. É preciso ter um equilíbrio entre a contribuição dos modelos e o número de árvores no conjunto.
<br>

- n_estimators: Como as árvores de decisão usadas no algoritmo são simples. O número de árvores adicionadas ao modelo deve ser alto funcione bem. O número de árvores o padrão é 50.
<br>

- base_estimator: Define o tipo de algoritmo utilizado no AdaBoost, o default é DecisionTreeClassifier(max_depth=1). Contudo é possível utilizarLogisticRegression() ou KNeighborsClassifier ().

<center><strong><font color = "darkblue" size=4> -----------------  ADA: Seleção de Hiperparâmetros -------------------</font></strong></center>

In [ ]:
# https://educationalresearchtechniques.com/2019/01/02/adaboost-classification-in-python/

test3 = { 'learning_rate':[0.1, 0.05, 0.01], 
            'n_estimators':[100, 250, 500, 1000,1500] }

ada    = AdaBoostClassifier()
search = GridSearchCV(estimator = ada, param_grid = test3, scoring = 'accuracy', n_jobs = 1, cv = 5)
search.fit(X_under_train[features], y_under_train)
search.best_params_

<center><strong><font color = "darkblue" size=4> -----------------  ADA: Dados Balanceados -------------------</font></strong></center>

In [ ]:
#https://www.datacamp.com/community/tutorials/adaboost-classifier-python?utm_source=adwords_ppc&utm_campaignid=10267161064&utm_adgroupid=102842301792&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377086&utm_targetid=aud-522010995285:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1001634&gclid=Cj0KCQjwgJv4BRCrARIsAB17JI76HKOOHYyhkIbR3eQUd9yfERhL7QoLCSehhj1eeEyyOV0cmVb5pRcaAlpnEALw_wcB

ada_under = AdaBoostClassifier(n_estimators = 500, learning_rate = 0.1).fit(X_under_train[features], y_under_train)

ML_IMPORTANCIA(ada_under, features)

# Train Adaboost Classifer
ada_test_under = AVALIA_MOD(modelo = ada_under, y_prob = X_test[features], y_obs = y_test, titulo = "AdaBo: Balanc", base = "Teste", corte =0.3)
ada_val_under  = AVALIA_MOD(modelo = ada_under, y_prob = X_val[features],  y_obs = y_val,  titulo = "AdaBo: Balanc", base = "Valida", corte = 0.3)

ada_test_under = SHIFT_AUC(x = ada_test_under, y = ada_val_under)

<center><strong><font color = "darkblue" size=4> -----------------  ADA: Dados Não Balanceados -------------------</font></strong></center>

In [ ]:
ada = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1).fit(X_train[features], y_train)

ML_IMPORTANCIA(ada, features)

# Train Adaboost Classifer
ada_test = AVALIA_MOD(modelo = ada, y_prob = X_test[features], y_obs = y_test, titulo = "AdaBo: Desbalanc", base = "Teste", corte = 0.3)
ada_val  = AVALIA_MOD(modelo = ada, y_prob = X_val[features],  y_obs = y_val,  titulo = "AdaBo: Desbalanc", base = "Valida", corte = 0.3)

ada_test = SHIFT_AUC(x = ada_test, y = ada_val)

<center><strong><font color = "darkblue" size=4> -----------------  ADA: Dados WOE Balanceados -------------------</font></strong></center>

In [ ]:
ada_WOE_under = AdaBoostClassifier(n_estimators = 500, learning_rate = 0.1).fit(X_under_train[X_WOE3], y_under_train)

ML_IMPORTANCIA(ada_WOE_under, X_WOE3)

# Train Adaboost Classifer
ada_test_WOE_under = AVALIA_MOD(modelo = ada_WOE_under, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "AdaBo: WOE Balanc", base = "Teste", corte = 0.5)
ada_val_WOE_under  = AVALIA_MOD(modelo = ada_WOE_under, y_prob = X_val[X_WOE3],  y_obs = y_val,  titulo = "AdaBo: WOE Balanc", base = "Valida", corte = 0.5)

ada_test_WOE_under = SHIFT_AUC(x = ada_test_WOE_under, y = ada_val_WOE_under)

<center><strong><font color = "darkblue" size=4> -----------------  ADA: Dados WOE Não Balanceados -------------------</font></strong></center>

In [ ]:
ada_WOE = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1).fit(X_train[X_WOE3], y_train)

ML_IMPORTANCIA(ada_WOE, X_WOE3)

# Train Adaboost Classifer
ada_WOE_test = AVALIA_MOD(modelo = ada_WOE, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "AdaBo: WOE Desbalanc", base = "Teste", corte = 0.5)
ada_WOE_val  = AVALIA_MOD(modelo = ada_WOE, y_prob = X_val[X_WOE3],  y_obs = y_val,  titulo = "AdaBo: WOE Desbalanc", base = "Valida", corte = 0.5)

ada_WOE_test = SHIFT_AUC(x = ada_WOE_test, y = ada_WOE_val)

---

<center><strong><font color = "darkblue" size=4> Redes Neurais </font></strong></center>

---

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

<center><strong><font color = "darkblue" size=4> -----------------  Redes Neurais: Ajuste hiperparâmetros -------------------</font></strong></center>

### Principais Parâmetros das Redes Neurais

- hidden_layer_sizes: Nos permite definir o número de camadas e o número de nós que desejamos ter. Cada elemento da tupla representa o número de nós na i-ésima posição em que i é o índice da tupla. Assim, o comprimento da tupla indica o número total de camadas ocultas na rede.
<br>

- activation: As funções de ativação introduzem o componente não linear nas redes neurais, que faz com que elas possam aprender mais do que relações lineares entre as variáveis dependentes e independentes, temos:
    - logistic, função sigmóide ou logística, retorna f(x) = 1/(1 + exp (-x))
    - tanh, função hiperbólica, retorna f(x) = tanh (x).
    - relu, retorna f(x) = max (0, x)
<br>

- solver: Especifica o algoritmo de otimização de peso entre os nós.
<br>

- alpha: É um parâmetro de regularização, também conhecido como termo de penalidade, que combate o excesso de ajuste restringindo o tamanho dos pesos. O aumento do alfa pode corrigir o overfitting, resultando em menor capacidade de molde aos dados. Da mesma forma, a diminuição do alpha pode corrigir um viés alto (underfitting), incentivando pesos maiores, resultando potencialmente em um limite de decisão mais complicado (overfitting).
<br>

- learning_rate: Controla o tamanho da etapa na atualização dos pesos. Usado somente quando solver = 'sgd' ou 'adam'.


In [ ]:
# https://datascience.stackexchange.com/questions/36049/how-to-adjust-the-hyperparameters-of-mlp-classifier-to-get-more-perfect-performa
# https://www.youtube.com/watch?v=tWKQ3Fk42yw
# https://matheusfacure.github.io/2017/07/12/activ-func/
# http://deeplearningbook.com.br/funcao-de-ativacao/#:~:text=ReLU%20%C3%A9%20a%20fun%C3%A7%C3%A3o%20de,neur%C3%B4nios%20ativados%20pela%20fun%C3%A7%C3%A3o%20ReLU.

mlp = MLPClassifier(max_iter=500, random_state = 999)

parameter_space = {
    'hidden_layer_sizes': [(2,3,5), (10,10,10), (10,25,50), (50,50,50), (50,100,150)],
    'activation': ['tanh', 'logistic','relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.01, 0.05],
    'learning_rate': ['constant','adaptive'],
}
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_under_train[X_WOE3], y_under_train)
clf.best_params_

<center><strong><font color = "darkblue" size=4> -----------------  Redes Neurais: Dados Balanceados -------------------</font></strong></center>

In [ ]:
RN_under = MLPClassifier(alpha = 0.05, hidden_layer_sizes = (10, 10, 10),
                         activation = "relu", learning_rate = "constant", solver = "adam",
                         max_iter = 1000).fit(X_under_train[features], y_under_train)


# Train Adaboost Classifer
rn_test_under = AVALIA_MOD(modelo = RN_under, y_prob = X_test[features], y_obs = y_test, titulo = "Redes: Balanc", base = "Teste", corte = 0.3)
rn_val_under  = AVALIA_MOD(modelo = RN_under, y_prob = X_val[features],  y_obs = y_val,  titulo = "Redes: Balanc", base = "Valida", corte = 0.3)

rn_test_under = SHIFT_AUC(x = rn_test_under, y = rn_val_under)

<center><strong><font color = "darkblue" size=4> -----------------  Redes Neurais: Dados Não Balanceados -------------------</font></strong></center>

In [ ]:
RN = MLPClassifier(alpha = 0.05, hidden_layer_sizes = (10, 20, 10),
                         activation = "relu", learning_rate = "constant", solver = "adam",
                         max_iter = 1000).fit(X_train[features], y_train)

# Train Adaboost Classifer
rn_test = AVALIA_MOD(modelo = RN, y_prob = X_test[features], y_obs = y_test, titulo = "Redes: Desbalanc", base = "Teste", corte = 0.5)
rn_val  = AVALIA_MOD(modelo = RN, y_prob = X_val[features],  y_obs = y_val,  titulo = "Redes: Desbalanc", base = "Valida", corte = 0.5)

rn_test = SHIFT_AUC(x = rn_test, y = rn_val)

<center><strong><font color = "darkblue" size=4> -----------------  Redes Neurais: Dados WOE Não Balanceados -------------------</font></strong></center>

In [ ]:
RN_WOE = MLPClassifier(alpha = 0.01, hidden_layer_sizes = (10, 10, 10),
                         activation = "relu", learning_rate = "constant", solver = "adam",
                         max_iter = 1000).fit(X_train[X_WOE3], y_train)

# Train Adaboost Classifer
rn_WOE_test = AVALIA_MOD(modelo = RN_WOE, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "Redes: WOE Desbalanc", base = "Teste", corte = np.mean(y_test))
rn_WOE_val  = AVALIA_MOD(modelo = RN_WOE, y_prob = X_val[X_WOE3],  y_obs = y_val,  titulo = "Redes: WOE Desbalanc", base = "Valida", corte = np.mean(y_val))

rn_WOE_test = SHIFT_AUC(x = rn_WOE_test, y = rn_WOE_val)

<center><strong><font color = "darkblue" size=4> -----------------  Redes Neurais: Dados WOE Balanceados -------------------</font></strong></center>

In [ ]:
RN_under_WOE = MLPClassifier(alpha = 0.01, hidden_layer_sizes = (10, 25, 50),
                         activation = "logistic", learning_rate = "constant", solver = "adam",
                         max_iter = 1000).fit(X_under_train[X_WOE3], y_under_train)

#ML_IMPORTANCIA(RN_under, features)

# Train Adaboost Classifer
rn_WOE_under_test = AVALIA_MOD(modelo = RN_under_WOE, y_prob = X_test[X_WOE3], y_obs = y_test, titulo = "Redes: WOE Balanc", base = "Teste", corte = 0.3)
rn_WOE_under_val  = AVALIA_MOD(modelo = RN_under_WOE, y_prob = X_val[X_WOE3],  y_obs = y_val,  titulo = "Redes: WOE Balanc", base = "Valida", corte = 0.3)

rn_WOE_under_test = SHIFT_AUC(x = rn_WOE_under_test, y = rn_WOE_under_val)

---

<center><strong><font color = "darkblue" size=4> Comparação dos modelos </font></strong></center>

---

In [ ]:
pdList = [
    
logit_under_WOE_test[2],
logit_WOE_test[2].iloc[:,1],
logit_under_test[2].iloc[:,1],
logit_test[2].iloc[:,1],    
    
rf_test_under_WOE[2].iloc[:,1],
rf_test_WOE[2].iloc[:,1],
rf_test_under[2].iloc[:,1],    
rf_test[2].iloc[:,1],
    
gb_under_WOE_test[2].iloc[:,1],  
gb_WOE_test[2].iloc[:,1],
gb_test_under[2].iloc[:,1], 
gb_test[2].iloc[:,1],
    
ada_WOE_test[2].iloc[:,1],
ada_test_WOE_under[2].iloc[:,1],    
ada_test_under[2].iloc[:,1],
ada_test[2].iloc[:,1],
    
rn_WOE_under_test[2].iloc[:,1],
rn_WOE_test[2].iloc[:,1],
rn_test_under[2].iloc[:,1],
rn_test[2].iloc[:,1]
    
]

In [ ]:
compara = pd.concat(pdList, axis=1)
compara.columns = ["Metricas", "Logit_WOE_balanc", "Logit_WOE", "Logit_balanc", "Logit", 
                               "RF_WOE_balanc", "RF_WOE", "RF_balanc", "RF",
                               "GB_WOE_balanc", "GB_WOE", "GB_balanc", "GB",
                               "ADA_WOE_balanc","ADA_WOE","ADA_balanc","ADA",
                               "NR_WOE_balanc","NR_WOE","NR_balanc","NR"
                  ]

In [ ]:
compara

In [ ]:
plot_ord = make_subplots(rows = 5, cols = 4,
                        subplot_titles = (
            logit_under_WOE_test[5],
            logit_WOE_test[5],
            logit_under_test[5],
            logit_test[5] ,
            rf_test_under_WOE[5],
            rf_test_WOE[5],
            rf_test_under[5],  
            rf_test[5],
            gb_under_WOE_test[5],
            gb_WOE_test[5],
            gb_test_under[5],
            gb_test[5],
            ada_test_WOE_under[5],   
            ada_WOE_test[5],
            ada_test_under[5],
            ada_test[5],
            rn_WOE_under_test[5] , 
            rn_WOE_test[5],
            rn_test_under[5],
            rn_test[5]                            
))

trace11 = logit_under_WOE_test[3]  
trace12 = logit_WOE_test[3]
trace13 = logit_under_test[3]
trace14 = logit_test[3]  

trace21 = rf_test_under_WOE[3]  
trace22 = rf_test_WOE[3]
trace23 = rf_test_under[3]
trace24 = rf_test[3]  

trace31 = gb_under_WOE_test[3]  
trace32 = gb_WOE_test[3]
trace33 = gb_test_under[3]
trace34 = gb_test[3]  

trace41 = ada_test_WOE_under[3]  
trace42 = ada_WOE_test[3]
trace43 = ada_test_under[3]
trace44 = ada_test[3]  

trace51 = rn_WOE_under_test[3]  
trace52 = rn_WOE_test[3]
trace53 = rn_test_under[3]
trace54 = rn_test[3] 

plot_ord.append_trace(trace11, 1, 1)
plot_ord.append_trace(trace12, 1, 2)
plot_ord.append_trace(trace13, 1, 3)
plot_ord.append_trace(trace14, 1, 4)

plot_ord.append_trace(trace21, 2, 1)
plot_ord.append_trace(trace22, 2, 2)
plot_ord.append_trace(trace23, 2, 3)
plot_ord.append_trace(trace24, 2, 4)

plot_ord.append_trace(trace31, 3, 1)
plot_ord.append_trace(trace32, 3, 2)
plot_ord.append_trace(trace33, 3, 3)
plot_ord.append_trace(trace34, 3, 4)

plot_ord.append_trace(trace41, 4, 1)
plot_ord.append_trace(trace42, 4, 2)
plot_ord.append_trace(trace43, 4, 3)
plot_ord.append_trace(trace44, 4, 4)

plot_ord.append_trace(trace51, 5, 1)
plot_ord.append_trace(trace52, 5, 2)
plot_ord.append_trace(trace53, 5, 3)
plot_ord.append_trace(trace54, 5, 4)

plot_ord.update_layout(height = 800, width = 1000, showlegend = False, title="Ordenação por Lift")
plot_ord.show()

In [ ]:

trace = {}
clrs  = {}

clrred  = "#73264d"
clrblue = "#00ace6"

for j in range(0,8):
    clrs[j] = [clrred if compara.iloc[j,1:][x] == max(compara.iloc[j,1:]) else clrblue for x in range(0, 20)]

clrs[7] = "#2d2d86" 
clrs[8] = "#2d2d86"  
    
nomes = [compara.iloc[l,0] for l in range(0,9)]  
nomes
fig_comp = make_subplots(rows=10, cols=1, subplot_titles = nomes)
for k in range(0,9):
    trace[k,0] = go.Scatter(x = compara.iloc[0,1:].reset_index().iloc[:,0], 
                            y = compara.iloc[k,1:], mode='markers',
                             marker_size = np.repeat(20,20), marker = dict(color = clrs[k]))
    
    
    fig_comp.append_trace(trace[k,0], k+1, 1)
    
fig_comp.update_layout(height = 2500, width = 900, showlegend = False, title="Medidas de qualidade dos modelos")

fig_comp.show()


In [ ]:
# Normalizando os dados usando Min-Max:
compara2 = compara.iloc[[0,3,4,5,6],1:].apply(lambda x: (x - min(x))/(max(x)-min(x)), axis=1)
aux_comp = compara2.apply(np.sum, axis=0).sort_values(ascending=False).reset_index()
aux_comp

fig = go.Figure(data=go.Scatter(x=aux_comp.iloc[:,0], y=aux_comp.iloc[:,1], mode="markers",
                               marker_size = np.repeat(30,20)))

fig.update_layout(height = 400, width = 900, showlegend = False, title="Rank dos modelos")
fig.show()